## Spark Structured Streaming - Read from Socket

In [1]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming Socket Word Count") \
    .master("local[*]") \
    .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/04 07:46:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Create the streaming dataframe to read from socket
# Sockets are not recommended for Production applications is only for debugging and testing applications
streaming_df = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", "9999") \
    .load()

23/01/04 07:46:32 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [3]:
# Check the schema
streaming_df.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
# Lets split the strings based on spaces and explode the list to create words column
words_df = streaming_df.selectExpr("explode(split(value, ' ')) as word")

# Check the schema
words_df.printSchema()

root
 |-- word: string (nullable = false)



In [5]:
# Now lets aggregate the words_df to find the word counts
from pyspark.sql.functions import count

# Change the shuffle partitions to 4 as we dont want to run through 200 partitions
spark.conf.set("spark.sql.shuffle.partitions", 4)

# Generate aggregated dataframe for word count
agg_words_df = words_df \
    .groupBy("word") \
    .agg(count("word").alias("count"))

# Print the schema to validate
agg_words_df.printSchema()

root
 |-- word: string (nullable = false)
 |-- count: long (nullable = false)



In [ ]:
# Write the output to console sink to check the output
writing_df = agg_words_df.writeStream \
    .format("console") \
    .outputMode("update") \
    .start()

# Start the streaming application to run until the following happens
# 1. Exception in the running program
# 2. Manual Interruption
writing_df.awaitTermination()

23/01/04 08:14:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-12cc23c2-ed68-4565-995e-e4225b3fbeb6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/04 08:14:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    1|
|spark|    1|
+-----+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+---------+-----+
|     word|count|
+---------+-----+
|    spark|    2|
|streaming|    1|
+---------+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    2|
|world|    1|
+-----+-----+

